In [1]:
%load_ext autoreload
%autoreload 2

In [71]:
import sys
sys.path.append('./scrapers/')

import datetime as dt
import json
from bson import json_util

from bs4 import BeautifulSoup
import pandas as pd

import pytz
from collections import defaultdict

In [182]:
import copy

def fltr(node, vals):
    if isinstance(node, dict):
        retVal = {}
        for key in node:
            if key in vals:
                retVal[key] = copy.deepcopy(node[key])
            elif isinstance(node[key], list) or isinstance(node[key], dict):
                child = fltr(node[key], vals)
                if child:
                    retVal[key] = child
        if retVal:
             return retVal
        else:
             return None
    elif isinstance(node, list):
        retVal = []
        for entry in node:
            child = fltr(entry, vals)
            if child:
                retVal.append(child)
        if retVal:
            return retVal
        else:
            return None

def preview(dataset):
    subset = []
    for publisher, scrape in article_dataset.items():
        for date, articles in scrape.items():
            for url, article in articles.items():
                article['date'] = date
                article['publisher'] = publisher

                article['scrape'] = {k: v for k, v in article['scrape'].items() if not k.endswith('html')}
                article['scrape']['paragraphs'] = article['scrape'].get('paragraphs', [])[:10]
                subset.append({k:v for k,v in article.items() if not k.endswith('html')})

    #             break
    #         break
    #     break

    print(json.dumps(subset, indent=4, default=json_util.default))
    return subset

def articles_to_df(dataset):
    records = []
    for publisher, scrape in article_dataset.items():
        for date, articles in scrape.items():
            for url, article in articles.items():
                rc = {}
                rc['date'] = date
                rc['publisher'] = publisher
                rc['url'] = url

                scrape = {k: v for k, v in article['scrape'].items() if not k.endswith('html')}
                
                paragraphs = scrape.get('paragraphs', [])#[:3]
                scrape['paragraphs'] = '\n'.join(paragraphs)
                rc.update(scrape)
                
                rc['rank'] = article['rank']
                rc['wayback_url'] = article['href']
                
                records.append(rc)
                
#                 subset.append({k:v for k,v in article.items() if not k.endswith('html')})
    return pd.DataFrame.from_records(records)


In [6]:
sampled_days_fp = '../analysis/production_run/sampled_days_36.tsv'
sampled_days = pd.read_csv(sampled_days_fp, sep='\t', index_col=0)
sampled_days

,day_int,date
0,0,2020-01-01
1,12,2020-01-13
2,22,2020-01-23
3,37,2020-02-07
4,40,2020-02-10
5,51,2020-02-21
6,66,2020-03-07
7,79,2020-03-20
8,83,2020-03-24
9,98,2020-04-08


In [21]:
from abcnews import ABCNewsScraper
scraper = ABCNewsScraper(verbose = True)

# from bbc import BBCScraper
# scraper = BBCScraper(verbose = True)

# from breitbart import BreitbartScraper
# scraper = BreitbartScraper(verbose = True)

# from foxnews import FoxnewsScraper
# scraper = FoxnewsScraper(verbose = True)

# from huffpost import HuffpostScraper
# scraper = HuffpostScraper(verbose = True)

# from nbcnews import NBCNewsScraper
# scraper = NBCNewsScraper(verbose = True)

# from nypost  import NYPostScraper
# scraper = NYPostScraper(verbose = True)

# from nytimes import NYTimesScraper
# scraper = NYTimesScraper(verbose = True)

# from theguardian import TheGuardianScraper
# scraper = TheGuardianScraper(verbose = True)

# from vox import VoxScraper
# scraper = VoxScraper(verbose = True)

# from washingtonpost import WashingtonpostScraper
# scraper = WashingtonpostScraper(verbose = True)

# from wsj import WSJScraper
# scraper = WSJScraper(verbose = True)

# from usatoday import USATodayScraper
# scraper = USATodayScraper(verbose = True)

In [23]:
article_dataset = defaultdict(dict)

In [170]:
init_scrape_hour = 18
est_timezone = pytz.timezone("US/Eastern")

start_idx = 29
for idx, date_str in enumerate(sampled_days['date'][start_idx:]):
    date = dt.datetime.strptime(date_str, '%Y-%m-%d')
    date += dt.timedelta(hours=init_scrape_hour)
    date = est_timezone.localize(date)
        
    print(start_idx + idx, date)
    
    articles = scraper.extract(date)
    
    publisher = scraper.front_page_url
    article_dataset[publisher][date] = articles
    
#     break
    

29 2020-10-26 18:00:00-04:00
Different domain https://web.archive.org/web/20201026220040/http://projects.fivethirtyeight.com/non-voters-poll-2020-election/
['https://abcnews.go.com/Politics/senate-republicans-poised-confirm-amy-coney-barrett-supreme/story?id=73792820', 'https://abcnews.go.com/Politics/wireStory/us-stocks-tumble-worst-day-month-amid-virus-73840886', 'https://abcnews.go.com/Politics/wireStory/police-florida-man-stole-bulldozer-ran-biden-signs-73833210', 'https://abcnews.go.com/Politics/wireStory/2020-watch-biden-remaking-democratic-coalition-73830034', 'https://abcnews.go.com/GMA/News/nxivm-founder-keith-raniere-face-sentencing/story?id=73838533']
https://abcnews.go.com/Politics/senate-republicans-poised-confirm-amy-coney-barrett-supreme/story?id=73792820
https://abcnews.go.com/Politics/wireStory/us-stocks-tumble-worst-day-month-amid-virus-73840886
https://abcnews.go.com/Politics/wireStory/police-florida-man-stole-bulldozer-ran-biden-signs-73833210
https://abcnews.go.com

In [190]:
pd.set_option('display.max_colwidth', 100)
df = articles_to_df(article_dataset)
df['date'] = df['date'].dt.tz_localize(None)
df['archive_time'] = df['archive_time'].dt.tz_localize(None)

df.to_excel('./abc_preview_scrape.xlsx')

# Debug Station
1. Can we reproduce the error?
2. Is it an archive / timestamp error?
3. Can you access it on the website?

In [143]:
url = 'https://abcnews.go.com/International/us-issues-warning-companies-importing-xinjiang-china/story?id=71559507'
time_str = "20200705013148"
timestamp = dt.datetime.strptime(time_str, "%Y%m%d%H%M%S")

scraper.get_wayback_url(url, timestamp)

('\n        <!doctype html>\n        <html lang="en">\n            <head><script src="//archive.org/includes/analytics.js?v=cf34f82" type="text/javascript"></script>\n<script type="text/javascript">window.addEventListener(\'DOMContentLoaded\',function(){var v=archive_analytics.values;v.service=\'wb\';v.server_name=\'wwwb-app102.us.archive.org\';v.server_ms=454;archive_analytics.send_pageview({});});</script><script type="text/javascript" src="/_static/js/playback.bundle.js?v=xTFGO54E" charset="utf-8"></script>\n<script type="text/javascript" src="/_static/js/wombat.js?v=i68pdzR-" charset="utf-8"></script>\n<script type="text/javascript">\n  __wm.init("https://web.archive.org/web");\n  __wm.wombat("https://abcnews.go.com/International/us-issues-warning-companies-importing-xinjiang-china/story?id=71559507","20200705014703","https://web.archive.org/","web","/_static/",\n\t      "1593913623");\n</script>\n<link rel="stylesheet" type="text/css" href="/_static/css/banner-styles.css?v=omkqRug

## Pull page for change in HTML

In [144]:
url = 'https://www.abcnews.go.com/'

start_idx = 29 
date_str = sampled_days['date'][start_idx]
date = dt.datetime.strptime(date_str, '%Y-%m-%d')
date += dt.timedelta(hours=init_scrape_hour)
date = est_timezone.localize(date)
timestamp = date

print(timestamp)
front_page, front_archive = scraper.get_front_page_html(timestamp)

with open('./abc_{}.html'.format(date_str), 'w', encoding='utf-8') as fp:
    fp.write(front_page)
    

2020-10-26 18:00:00-04:00


In [169]:
# scraper.front_page_url.strip('(https://)|(www\.)')
articles = scraper.get_top_article_metadata(front_page)

Different domain https://web.archive.org/web/20201026220040/http://projects.fivethirtyeight.com/non-voters-poll-2020-election/
['https://abcnews.go.com/Politics/senate-republicans-poised-confirm-amy-coney-barrett-supreme/story?id=73792820', 'https://abcnews.go.com/Politics/wireStory/us-stocks-tumble-worst-day-month-amid-virus-73840886', 'https://abcnews.go.com/Politics/wireStory/police-florida-man-stole-bulldozer-ran-biden-signs-73833210', 'https://abcnews.go.com/Politics/wireStory/2020-watch-biden-remaking-democratic-coalition-73830034', 'https://abcnews.go.com/GMA/News/nxivm-founder-keith-raniere-face-sentencing/story?id=73838533']


In [168]:
from urllib.parse import urlparse

urlparse(scraper.front_page_url)
urlparse("http://projects.fivethirtyeight.com/non-voters-poll-2020-election/")

ParseResult(scheme='http', netloc='projects.fivethirtyeight.com', path='/non-voters-poll-2020-election/', params='', query='', fragment='')

In [179]:
list(article_dataset['https://www.abcnews.go.com/'].items())[0]

(datetime.datetime(2020, 1, 1, 18, 0, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>),
 OrderedDict([('https://abcnews.go.com/Politics/rudy-giuliani-testify-give-demonstrations-trump-impeachment-trial/story?id=68017920',
               {'rank': 1,
                'href': 'https://web.archive.org/web/20200101223229/https://abcnews.go.com/Politics/rudy-giuliani-testify-give-demonstrations-trump-impeachment-trial/story?id=68017920',
                'headline': "Giuliani willing to testify and give 'demonstrations' at impeachment trial",
                'html': {'headline': '<a class="black-ln" data-analytics="cid=clicksource_4380645_null_hero_hed" href="https://web.archive.org/web/20200101223229/https://abcnews.go.com/Politics/rudy-giuliani-testify-give-demonstrations-trump-impeachment-trial/story?id=68017920">\nGiuliani willing to testify and give \'demonstrations\' at impeachment trial\n</a>'},
                'scrape': {'html': '\n        <!doctype html>\n        <html lang="e

In [175]:
import pickle

with open('abc_scrape.pkl', 'wb') as fp:
    pickle.dump(article_dataset, fp)